In [1]:
import sys, os
import argparse

sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), "./../../../../")))
sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), "./../../../")))

sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), "")))

In [2]:
from fedml_api.data_preprocessing.cifar10.data_loader import load_partition_data_cifar10
from fedml_api.standalone.fedavg.my_model_trainer_classification import MyModelTrainer as MyModelTrainerCLS
from fedml_api.model.contrastive_cv.resnet_with_embedding import Resnet56
from CovaMNet import CovaMResnet56

import torch
from torch import nn
from collections import OrderedDict
import torch.nn.functional as F

import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
import numpy as np
import random
import pickle
import wandb

import torch.nn.functional as F

from pcgrad import PCGrad

# from triplet_loss import TripletLoss
from hard_triplet_loss import TripletLoss

<frozen importlib._bootstrap>:219: RuntimeWarning: greenlet.greenlet size changed, may indicate binary incompatibility. Expected 144 from C header, got 152 from PyObject
<frozen importlib._bootstrap>:219: RuntimeWarning: greenlet.greenlet size changed, may indicate binary incompatibility. Expected 144 from C header, got 152 from PyObject
<frozen importlib._bootstrap>:219: RuntimeWarning: greenlet.greenlet size changed, may indicate binary incompatibility. Expected 144 from C header, got 152 from PyObject
<frozen importlib._bootstrap>:219: RuntimeWarning: greenlet.greenlet size changed, may indicate binary incompatibility. Expected 144 from C header, got 152 from PyObject


In [3]:
dataset = 'cifar10'
data_dir = "./../../../data/cifar10"
partition_method = 'hetero'
# partition_method = 'homo'
partition_alpha = 100
client_num_in_total = 10
batch_size = 100
total_epochs = 500

save_model_path = 'model/cs_{0}_{1}_client_{2}_orcal_resnet_{3}.pt'

device = 'cuda:2'
with_cova = False
# -_0.2heterodist_reg_init_weights_inv_min_better_no_pcgrad_offline_cova
wandb.init(
            # project="federated_nas",
            project="IS_weight_training",
            name="Cova_with_IS_" +str(partition_alpha)+ "_weight" + str(partition_method) + "_clients" + str(client_num_in_total)
#             name="Cova_" + str(partition_method) + "_clients" + str(client_num_in_total) + "-_0.2heterodist_Mdist_reg_inv_min_better_no_pcgrad_offline_cova" + str(
#                 with_cova)
        )
# config = wandb.config
# config.learning_rate = 0.01

# with open(f'dataset_{partition_method}_{partition_alpha}_{client_num_in_total}.pickle', 'rb') as f:
with open(f'dataset_{partition_method}_{partition_alpha}_{client_num_in_total}_with_cls_counts.pickle', 'rb') as f:
    dataset = pickle.load(f)
# [train_data_num, test_data_num, train_data_global, test_data_global, \
# #             train_data_local_num_dict, train_data_local_dict, test_data_local_dict, \
# #             class_num]

ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: slimfun (use `wandb login --relogin` to force relogin)
wandb: wandb version 0.12.7 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


In [4]:
# train_data_num, test_data_num, train_data_global, test_data_global, \
# train_data_local_num_dict, train_data_local_dict, test_data_local_dict, \
# class_num = load_partition_data_cifar10(dataset, data_dir, partition_method,
#                         partition_alpha, client_num_in_total, batch_size)

In [5]:
# d = [train_data_num, test_data_num, train_data_global, test_data_global, \
# train_data_local_num_dict, train_data_local_dict, test_data_local_dict, \
# class_num]

# with open(f'dataset_{partition_method}_{partition_alpha}_{client_num_in_total}.pickle', 'wb') as f:
#     pickle.dump(d, f)

# from collections import Counter
# ls = np.array([int(l) for l in labels])
# counter_result = Counter(ls)
# print(Counter(ls))
# print(dataset[5])

In [6]:
# model = Resnet56(class_num=dataset[-1], neck='bnneck')
# model.load_state_dict(torch.load('model/cs_3_homo_client_0_orcal_resnet_200.pt'))

In [7]:
# def test(model, test_data, device):
#     model.to(device)
#     model.eval()

#     metrics = {
#         'test_correct': 0,
#         'test_loss': 0,
#         'test_total': 0
#     }

#     criterion = nn.CrossEntropyLoss().to(device)

#     with torch.no_grad():
#         for batch_idx, (x, target) in enumerate(test_data):
#             x = x.to(device)
#             target = target.to(device)
#             pred, feat = model(x)
#             loss = criterion(pred, target)

#             _, predicted = torch.max(pred, -1)
#             correct = predicted.eq(target).sum()

#             metrics['test_correct'] += correct.item()
#             metrics['test_loss'] += loss.item() * target.size(0)
#             metrics['test_total'] += target.size(0)
            
#     return metrics

# def cal_covariance(input):

#     CovaMatrix_list = []
#     mean_list = []
#     for i in range(len(input)):
#         support_set_sam = input[i]
#         support_set_sam = torch.unsqueeze(support_set_sam, 0)
#         B, C, h, w = support_set_sam.size()
        

#         support_set_sam = support_set_sam.permute(1, 0, 2, 3)
#         support_set_sam = support_set_sam.contiguous().view(C, -1)

#         mean_support = torch.mean(support_set_sam, 1, True)
#         mean_list.append(mean_support)

#         support_set_sam = support_set_sam-mean_support

#         covariance_matrix = support_set_sam@torch.transpose(support_set_sam, 0, 1)
#         covariance_matrix = torch.div(covariance_matrix, h*w*B-1)
#         CovaMatrix_list.append(covariance_matrix)

#     return CovaMatrix_list, mean_list
        
        

# cl = [torch.zeros((256,256)) for i in range(10)]
# ml = [torch.zeros((256,64)) for i in range(10)]
# lbd = 0.999
# labels = []
# def extract_features(model, data_loader, device):
#     model.to(device)
#     model.eval()
    
#     features = []
    
    
#     with torch.no_grad():
#         for batch_idx, (x, l) in enumerate(data_loader):
#             x, l = x.to(device), l.to(device)
            
#             score, feats = model(x)
#             covaM_list, mean_list = cal_covariance(feats)
#             for covaM, f, label in zip(covaM_list, feats, l):
#                 labels.append(label.cpu())
#                 for i in range(len(cl)):
#                     if label.data.cpu() == i:
#                         cl[i] = lbd * cl[i] + (1-lbd) * covaM.cpu()
#                         f = torch.unsqueeze(f, 0)
#                         B, C, h, w = f.size()

#                         f = f.permute(1, 0, 2, 3)
#                         f = f.contiguous().view(C, -1)
#                         ml[i] = lbd * ml[i] + (1-lbd) * f.cpu()

# metrics = test(model, dataset[3], device)
# test_correct = metrics['test_correct']/metrics['test_total']
# test_loss = metrics['test_loss']/metrics['test_total']
# print(metrics['test_total'])
# print(f'test_correct: {test_correct}; test_loss: {test_loss}')
# extract_features(model, dataset[2], device)
# covaMs_means = [cl, ml]
# with open(f'orca_l_covaMs_means.pickle', 'wb') as f:
#     pickle.dump(covaMs_means, f)

In [8]:
with open(f'better_orca_no_bnneck_l_covaMs_means.pickle', 'rb') as f:
# with open(f'better_orca_no_bnneck_l_covaMs_means.pickle', 'rb') as f:
    cms = pickle.load(f)
    
for i in range(len(cms[0])):
#     cms[0][i] = torch.eye(256).to(device) / 256
#     cms[0][i] = torch.rand((256,256)).to(device) / 10
#     cms[0][i] = cms[0][i].to(device)
    cms[0][i] = cms[0][i].inverse().to(device)
print(len(cms[0]))
# print(cms[0][0] - cl[0])
# print(cms[1][1] - ml[1])
# from collections import Counter
# ls = np.array([int(l) for l in labels])
# counter_result = Counter(ls)
# print(Counter(ls))
# Counter({1: 1145, 2: 1138, 0: 797, 3: 661, 4: 555, 5: 539, 8: 227, 7: 38})


10


In [9]:
class Client(object):
    def __init__(self, client_index, train_data_local_dict, train_data_local_num_dict, test_data_local_dict, traindata_cls_counts, device, model):
        self.id = client_index
        self.train_data = train_data_local_dict[self.id]
        self.local_sample_number = train_data_local_num_dict[self.id]
        self.test_local = test_data_local_dict[self.id]
        self.traindata_cls_counts = traindata_cls_counts[self.id]
        
        self.device = device
        self.model = model
# model1 = CovaMResnet56(class_num=dataset[7], neck='no', with_cova=with_cova)
model1 = Resnet56(class_num=dataset[7], neck='no')
client_1 = Client(0, dataset[5], dataset[4], dataset[6], dataset[-1], device, model1)

In [10]:
def test(model, test_data, device, epoch):
    cl = cms[0]
    ml = cms[1]
    
    model.to(device)
    model.eval()

    metrics = {
        'test_correct': 0,
        'test_loss': 0,
        'test_total': 0,
        'correct_cova': 0
    }

    criterion = nn.CrossEntropyLoss().to(device)

    with torch.no_grad():
        for batch_idx, (x, target) in enumerate(test_data):
            x = x.to(device)
            target = target.to(device)
            support_ml = []
            for m in ml:
                support_ml.append(m.to(device))
            pred, feat = model(x)
            loss = criterion(pred, target)
#             print(cls_score)
#             print(f'test: {pred}')
#             print(pred)
#             print(pred_ce_cova)
#             if epoch > 3:
#                 print(pred_ce_cova)
            
#             loss_ce_cova = criterion(pred_ce_cova, target)

            _, predicted = torch.max(pred, -1)
#             _, predicted_ce_cova = torch.max(pred_ce_cova, -1)
            correct = predicted.eq(target).sum()
#             correct_ce_cova = predicted_ce_cova.eq(target).sum()

            metrics['test_correct'] += correct.item()
            metrics['test_loss'] += loss.item() * target.size(0)
            metrics['test_total'] += target.size(0)
#             metrics['correct_cova'] += correct_ce_cova.item()
            
    return metrics

def train_model(client, epochs):
#     learning_rate = 0.001
#     wd = 0.0001
#     learning_rate = 0.00035
#     wd = 0.0005
    print(cms[0][0])
    cl = cms[0]
    ml = cms[1]
#     print(ml)
    
    margin = 0.3
    lbd = 2
    
#     a = torch.Tensor(ml)
#     a = torch.mean(torch.stack(ml), dim=2)
#     print(a.shape)
#     client.model.re_init_classifier(ml)
#     for key in client.model.state_dict().keys():
#         if key == 'ce_classifier.weight':
#             t = model1.state_dict()[key]
#             t.copy_(a)
#         if key == 'ce_classifier.bias':
#             t = model1.state_dict()[key]
#             t.copy_(torch.zeros(t.shape))
#     client.model.ce_classifier.requires_grad = False
#         if key == ''
    
    client.model.to(client.device)
    client.model.train()
    
    # [train_data_num, test_data_num, train_data_global, test_data_global, \
    # #             train_data_local_num_dict, train_data_local_dict, test_data_local_dict, \
    # #             class_num]
    p_local = []
    print(client.local_sample_number)
    print(client.traindata_cls_counts)
    for i in range(10):
        if i in client.traindata_cls_counts:
            p_local.append(client.traindata_cls_counts[i] / float(client.local_sample_number))
        else:
            p_local.append(0.0)
    print(p_local)
    
    criterion = nn.CrossEntropyLoss(reduction='none').to(device)
#     criterion2 = nn.CrossEntropyLoss().to(device)
    
#     curr_lr = learning_rate
#     optimizer = torch.optim.Adam(filter(lambda p: p.requires_grad, client.model.parameters()), lr=curr_lr,
#                                          weight_decay=wd, amsgrad=True)
    optimizer = torch.optim.SGD(client.model.parameters(), lr=0.1,
                      momentum=0.9, weight_decay=5e-4)
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=200)
#     optimizer = torch.optim.Adam(client.model.parameters(), lr=0.001, betas=(0.5, 0.9))
#     optimizer = PCGrad(torch.optim.Adam(client.model.parameters(), lr=0.001, betas=(0.5, 0.9)))
#     optimizer = PCGrad(torch.optim.Adam(client.model.parameters(), lr=0.001, betas=(0.5, 0.9)))

    epoch_loss = []
    total_step = len(client.train_data)
#     wandb.watch(client.model, log='all')
    for epoch in range(epochs):
        client.model.train()
        batch_loss = []
        for batch_idx, (x, labels) in enumerate(client.train_data):
            x, labels = x.to(device), labels.to(device)
#             print(labels.shape)
            client.model.zero_grad()
#             support_ml = []
#             for m in ml:
#                 support_ml.append(m.to(device))
            support_ml = []
            for l in labels:
                support_ml.append(ml[l.data].to(device))
            cls_score, feats = client.model(x)
#             print(cls_score)
            
#             print(cls_score)
#             support_ml = torch.stack(support_ml)
#             print(score.shape)
#             loss = criterion(cls_score, labels)
#             print(support_ml.shape)
#             print(feats)
#             B, C, h, w = feats.size()
#             feats = feats.view(B, C, -1)
#             if epoch > 5:
#             print(feats.sum())
#             print(cls_score)
#             distance = F.pairwise_distance(feat, support_ml, p=2)

#             feats = feats - support_ml
#             sum_dist = 0
# #             print(len(feats))
# #             B = len(feats)
#             for i in range(B):
#                 feat = feats[i]
#                 l = labels[i]
# #                 C, h, w = feat.size()
# #                 feat = feat.view(C, -1)
# # #                 print(B)
# # #                 print(feat.shape)
# # #                 print(support_ml[i].shape)
# #                 feat = feat - support_ml[i]
#                 temp_dis = torch.transpose(feat, 0, 1)@feat
#                 temp_dis = torch.transpose(feat, 0, 1)@cl[l.data]@feat
#                 diag = temp_dis.diag()
#                 sum_dist += diag.sum() / (len(diag))
#             sum_dist /= B
            
#             distance = torch.norm(feats - support_ml, p=2, dim=(1,2)).sum()/(B*h*w*C)
#             print(distance.shape)
#             loss = distance
#             print(distance)
            is_weights = []
            for l in labels:
                is_weights.append((1.0 / 10) / p_local[l.cpu()])
            is_weights = torch.FloatTensor(is_weights).to(device)
            loss = (is_weights * criterion(cls_score, labels)).mean()
#             loss = criterion(cls_score, labels)
#             loss = criterion(cls_score, labels) + distance
#             loss = loss + criterion(cova_score, labels)
#             loss = loss + distance
            
#             loss = criterion(cova_score, labels)
            
            loss.backward()
#             loss1 = criterion(cls_score, labels)
#             loss2 = criterion2(cova_score, labels)
#             losses = [loss1, loss2]
#             losses = [loss1]
#             optimizer.pc_backward(losses)
#             loss.backward()
        
#             torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
#             print(client.model.cova_score.grad)
#             for name, param in client.model.named_parameters():
#                 if param.requires_grad:
#                     print(f'{name} --> {param.grad}\n')
                    

            optimizer.step()
#             batch_loss.append(loss1.item())
            batch_loss.append(loss.item())
#             batch_loss.append((loss1+loss2).item())
            
#         lbd += (5.0-lbd)/200.0
#         if epoch % 50 == 0:
#             torch.save(client.model.state_dict(), str.format(save_model_path, client_num_in_total, partition_method, client.id, epoch))
        epoch_loss.append(sum(batch_loss) / len(batch_loss))
        scheduler.step()
        print('Client Index = {}\tEpoch: {}\tLoss: {:.6f}'.format(
            client.id, epoch, sum(batch_loss) / len(batch_loss)))
        
        metrics = test(client.model, client.test_local, client.device, epoch)
        test_correct = metrics['test_correct']/metrics['test_total']
        test_loss = metrics['test_loss']/metrics['test_total']
#         correct_ce_cova = metrics['correct_cova']/metrics['test_total']
#         print(metrics['test_total'])
        print(f'test_correct: {test_correct}; test_loss: {test_loss};')
        wandb.log({
#             'epoch' : epoch,
            'train_loss': sum(batch_loss) / len(batch_loss),
            'test_correct' : test_correct,
            'test_loss' : test_loss,
#             'correct_cova' : correct_ce_cova,
#             'lbd': lbd
        }, step=epoch)
        
    torch.save(client.model.state_dict(), str.format(save_model_path, client_num_in_total, partition_method, client.id, epochs))

In [11]:
train_model(client_1, 200)

tensor([[ 1.2002e+01, -6.0263e-01, -1.6839e-01,  ..., -3.7063e-01,
          1.4831e-01, -1.5194e+00],
        [-6.0263e-01,  9.9602e+00,  4.4257e-01,  ..., -4.1334e-01,
         -3.3424e-02,  1.4330e-01],
        [-1.6838e-01,  4.4257e-01,  8.2606e+00,  ...,  2.8520e-03,
          2.1124e-01, -5.2687e-01],
        ...,
        [-3.7063e-01, -4.1334e-01,  2.8517e-03,  ...,  1.1538e+01,
         -2.4612e-01,  6.3535e-01],
        [ 1.4831e-01, -3.3426e-02,  2.1124e-01,  ..., -2.4612e-01,
          6.7511e+00, -1.1571e-01],
        [-1.5194e+00,  1.4330e-01, -5.2687e-01,  ...,  6.3535e-01,
         -1.1571e-01,  1.6578e+01]], device='cuda:2')
4841
{0: 475, 1: 472, 2: 559, 3: 501, 4: 448, 5: 509, 6: 516, 7: 420, 8: 517, 9: 424}
[0.09812022309440198, 0.09750051642222682, 0.11547200991530676, 0.10349101425325347, 0.09254286304482545, 0.10514356537905391, 0.10658954761412931, 0.08675893410452386, 0.10679611650485436, 0.08758520966742409]
Client Index = 0	Epoch: 0	Loss: 3.243321
test_correct:

Client Index = 0	Epoch: 89	Loss: 0.822181
test_correct: 0.6588; test_loss: 1.1417795187234878;
Client Index = 0	Epoch: 90	Loss: 0.817011
test_correct: 0.6638; test_loss: 1.0316036933660506;
Client Index = 0	Epoch: 91	Loss: 0.808510
test_correct: 0.6299; test_loss: 1.1740903401374816;
Client Index = 0	Epoch: 92	Loss: 0.787762
test_correct: 0.6254; test_loss: 1.2102423280477523;
Client Index = 0	Epoch: 93	Loss: 0.772725
test_correct: 0.6866; test_loss: 1.0236373084783554;
Client Index = 0	Epoch: 94	Loss: 0.772443
test_correct: 0.666; test_loss: 1.1418560510873794;
Client Index = 0	Epoch: 95	Loss: 0.767692
test_correct: 0.5922; test_loss: 1.4205128681659698;
Client Index = 0	Epoch: 96	Loss: 0.777929
test_correct: 0.6734; test_loss: 0.9811268800497055;
Client Index = 0	Epoch: 97	Loss: 0.774899
test_correct: 0.6719; test_loss: 1.0174227571487426;
Client Index = 0	Epoch: 98	Loss: 0.743187
test_correct: 0.6259; test_loss: 1.304604018330574;
Client Index = 0	Epoch: 99	Loss: 0.742174
test_corre

Client Index = 0	Epoch: 175	Loss: 0.139774
test_correct: 0.7658; test_loss: 0.9618407225608826;
Client Index = 0	Epoch: 176	Loss: 0.138513
test_correct: 0.7641; test_loss: 0.9677769649028778;
Client Index = 0	Epoch: 177	Loss: 0.130151
test_correct: 0.7687; test_loss: 0.9473081594705581;
Client Index = 0	Epoch: 178	Loss: 0.141890
test_correct: 0.7704; test_loss: 0.9641659837961197;
Client Index = 0	Epoch: 179	Loss: 0.122001
test_correct: 0.7738; test_loss: 0.9457905739545822;
Client Index = 0	Epoch: 180	Loss: 0.121800
test_correct: 0.7669; test_loss: 0.9621596407890319;
Client Index = 0	Epoch: 181	Loss: 0.120119
test_correct: 0.7697; test_loss: 0.9572308444976807;
Client Index = 0	Epoch: 182	Loss: 0.121505
test_correct: 0.77; test_loss: 0.9579255658388138;
Client Index = 0	Epoch: 183	Loss: 0.123373
test_correct: 0.7716; test_loss: 0.9449875217676162;
Client Index = 0	Epoch: 184	Loss: 0.125420
test_correct: 0.7723; test_loss: 0.9434639322757721;
Client Index = 0	Epoch: 185	Loss: 0.114823

In [12]:
# 0.5: root:traindata_cls_counts = {0: {0: 844, 1: 45, 2: 196, 3: 50, 4: 8, 5: 437, 6: 3, 7: 1, 8: 206, 9: 3811}, /5601
#                              1: {0: 505, 1: 1993, 2: 979, 3: 13, 4: 1683}, 2: {0: 29, 1: 255, 2: 447, 3: 405, 4: 527, 5: 925, 6: 206, 7: 6, 8: 353, 9: 604}, 3: {0: 863, 1: 23, 2: 178, 3: 3127, 4: 76, 5: 4, 6: 231, 7: 524}, 4: {0: 334, 1: 22, 2: 701, 3: 491, 4: 278, 5: 2736, 6: 1107}, 5: {0: 839, 1: 683, 2: 5, 3: 172, 4: 458, 5: 763, 6: 88, 7: 694, 8: 15, 9: 15}, 6: {0: 462, 1: 99, 2: 1513, 3: 4, 4: 754, 5: 70, 6: 2376}, 7: {0: 203, 1: 273, 2: 681, 3: 709, 4: 1213, 5: 19, 6: 242, 7: 2694}, 8: {0: 723, 1: 1485, 2: 263, 3: 6, 4: 1, 5: 4, 6: 489, 7: 762, 8: 889, 9: 470}, 9: {0: 198, 1: 122, 2: 37, 3: 23, 4: 2, 5: 42, 6: 258, 7: 319, 8: 3537, 9: 100}}

# homo: {0: {0: 496, 1: 459, 2: 523, 3: 488, 4: 531, 5: 508, 6: 497, 7: 502, 8: 510, 9: 486}

# 100: {0: {0: 475, 1: 472, 2: 559, 3: 501, 4: 448, 5: 509, 6: 516, 7: 420, 8: 517, 9: 424}

def count_label_num(dataset):
    labels = [0 for i in range(10)]
    for batch_idx, (x, labels) in enumerate(client.train_data):
        for l in labelsels:
            l = int(l.data)
            labels[l] += 1
    return labels

